In [ ]:
import pandas as pd

poem = pd.read_csv("唐詩三百首.csv")
poem_texts = poem['Poem Text'].to_list()
print(poem_texts)

In [5]:
unique_chars = set()

for text in poem_texts:
    text_chars = set(text)
    unique_chars.update(text_chars)

print(unique_chars)
count_unique_chars = len(unique_chars)
print(count_unique_chars)

{'惡', '夏', '旄', '秩', '廈', '俗', '葵', '霄', '妙', '攢', '宅', '五', '煮', '玦', '藍', '悅', '蛟', '噴', '冰', '鼎', '量', '走', '成', '彩', '習', '鳥', '冠', '宇', '媚', '刻', '怪', '兼', '儒', '恭', '胡', '疾', '折', '藏', '術', '語', '狗', '降', '矛', '鵲', '漢', '戶', '吐', '寄', '苑', '毅', '偏', '縞', '衾', '慰', '限', '簷', '壺', '飆', '寺', '趨', '垂', '萄', '泛', '削', '通', '頭', '蟢', '鄭', '小', '臼', '牛', '洲', '臂', '升', '腥', '聞', '肝', '動', '祝', '岧', '開', '接', '洮', '所', '埋', '殿', '辨', '駕', '波', '穆', '淡', '婦', '浦', '手', '潘', '蟾', '瓢', '圍', '弱', '剜', '工', '任', '上', '飫', '坤', '格', '陋', '於', '似', '闇', '雌', '鐘', '唯', '謳', '太', '勇', '摩', '排', '揀', '禁', '幸', '神', '記', '妨', '畔', '矣', '坎', '豺', '州', '及', '死', '靜', '谷', '仰', '肅', '帷', '隘', '福', '租', '泗', '絕', '啄', '傅', '響', '嫦', '坼', '北', '篇', '蝦', '槐', '卿', '霓', '蒲', '起', '蔽', '圖', '窕', '獨', '奠', '勤', '塔', '初', '競', '短', '松', '眸', '彼', '條', '美', '抽', '襪', '擣', '呂', '銖', '擬', '怒', '忘', '田', '想', '幼', '傾', '羿', '家', '垢', '靂', '物', '堂', '泯', '渾', '冥', '嬪', '耳', '逢', '射', '恣', '呵', '翼', '滋', '儉', '燈',

## Approach: statistical counting

In [1]:
import torch

In [6]:
N = torch.zeros((2532, 2532), dtype=torch.int32)

In [10]:
stoi['。']

1

In [18]:
chars = sorted(list(set(''.join(poem_texts))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

for w in poem_texts:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N[ix1, ix2] += 1

P = N.float()
P /= P.sum(1, keepdims=True)
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
  
  out = []
  ix = 0
  while True:
    p = P[ix]
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

漢朝河倒，夜，東流水蕩不得瞿塘。.
漢宮。登太華重，但道，門上，江上寒齒今宵。山安見五更遲暮年多。周綱陵。盤孔鼎有淥水迢遞三日望空憶？一杯消萬世法宮花川帶長風送客楚角聲。烽火對門，倚闌干。行歸青鳥性命惟案几。.
八代衣。廬，寒。田里黃，去，得。閒坐相頭白蘋。.
晚。.
劍動倒，妾夢，今。道哭，空死生白雲峰長風送八千秋水無人歇待周。丹青樓。玉京洛陽，念益渺然中有一枝。沙頭歇，嘉客去天都門令尹誰教明思正肥，若有以精。天地曲似雪浮。.


## Approach: neural net


In [39]:
import torch

In [40]:
chars = sorted(list(set(''.join(poem_texts))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [41]:
# create the training set of bigrams (x,y)
xs, ys = [], []

for w in poem_texts[:1]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    print(ch1, ch2)
    xs.append(ix1)
    ys.append(ix2)
    
xs = torch.tensor(xs)
ys = torch.tensor(ys)

. 昔
昔 歲
歲 逢
逢 太
太 平
平 ，
， 山
山 林
林 二
二 十
十 年
年 。
。 泉
泉 源
源 在
在 庭
庭 戶
戶 ，
， 洞
洞 壑
壑 當
當 門
門 前
前 。
。 井
井 稅
稅 有
有 常
常 期
期 ，
， 日
日 晏
晏 猶
猶 得
得 眠
眠 。
。 忽
忽 然
然 遭
遭 世
世 變
變 ，
， 數
數 歲
歲 親
親 戎
戎 旃
旃 。
。 今
今 來
來 典
典 斯
斯 郡
郡 ，
， 山
山 夷
夷 又
又 紛
紛 然
然 。
。 城
城 小
小 賊
賊 不
不 屠
屠 ，
， 人
人 貧
貧 傷
傷 可
可 憐
憐 。
。 是
是 以
以 陷
陷 鄰
鄰 境
境 ，
， 此
此 州
州 獨
獨 得
得 全
全 。
。 使
使 臣
臣 將
將 王
王 命
命 ，
， 豈
豈 不
不 如
如 賊
賊 焉
焉 。
。 令
令 彼
彼 徵
徵 歛
歛 者
者 ，
， 迫
迫 之
之 如
如 火
火 煎
煎 。
。 誰
誰 能
能 絕
絕 人
人 命
命 ，
， 以
以 作
作 時
時 世
世 賢
賢 。
。 思
思 欲
欲 委
委 符
符 節
節 ，
， 引
引 竿
竿 自
自 刺
刺 船
船 。
。 將
將 家
家 就
就 魚
魚 麥
麥 ，
， 歸
歸 老
老 江
江 湖
湖 邊
邊 。
。 .


In [42]:
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=2531).float()
xenc

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.]])

In [43]:
xenc.shape

torch.Size([145, 2531])

In [47]:
W = torch.randn((2531, 2531))
logits = xenc @ W # log-counts
counts = logits.exp() # equivalent N
probs = counts / counts.sum(1, keepdims=True)
probs

tensor([[1.8690e-04, 9.7821e-04, 6.6930e-05,  ..., 2.6822e-03, 1.4612e-04,
         7.2054e-04],
        [1.6789e-04, 6.8892e-04, 1.1538e-04,  ..., 1.4839e-04, 8.2009e-03,
         3.7310e-04],
        [5.9519e-05, 1.3934e-04, 1.7157e-03,  ..., 6.3312e-04, 1.7903e-04,
         7.3833e-05],
        ...,
        [2.8745e-04, 1.3360e-04, 2.4207e-04,  ..., 1.0472e-04, 1.1362e-04,
         3.6471e-05],
        [9.3967e-05, 1.5334e-04, 2.0669e-03,  ..., 1.6180e-04, 2.8349e-03,
         8.1469e-04],
        [3.9970e-04, 2.8634e-04, 4.6906e-05,  ..., 9.2421e-04, 9.6243e-04,
         6.7610e-04]])

In [48]:
probs.shape

torch.Size([145, 2531])

In [49]:
nlls = torch.zeros(5)
for i in range(5):
  # i-th bigram:
  x = xs[i].item() # input character index
  y = ys[i].item() # label character index
  print('--------')
  print(f'bigram example {i+1}: {itos[x]}{itos[y]} (indexes {x},{y})')
  print('input to the neural net:', x)
  print('output probabilities from the neural net:', probs[i])
  print('label (actual next character):', y)
  p = probs[i, y]
  print('probability assigned by the net to the the correct character:', p.item())
  logp = torch.log(p)
  print('log likelihood:', logp.item())
  nll = -logp
  print('negative log likelihood:', nll.item())
  nlls[i] = nll

print('=========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())

--------
bigram example 1: .昔 (indexes 0,955)
input to the neural net: 0
output probabilities from the neural net: tensor([1.8690e-04, 9.7821e-04, 6.6930e-05,  ..., 2.6822e-03, 1.4612e-04,
        7.2054e-04])
label (actual next character): 955
probability assigned by the net to the the correct character: 0.0002630152739584446
log likelihood: -8.243298530578613
negative log likelihood: 8.243298530578613
--------
bigram example 2: 昔歲 (indexes 955,1124)
input to the neural net: 955
output probabilities from the neural net: tensor([0.0002, 0.0007, 0.0001,  ..., 0.0001, 0.0082, 0.0004])
label (actual next character): 1124
probability assigned by the net to the the correct character: 0.00013051970745436847
log likelihood: -8.943985939025879
negative log likelihood: 8.943985939025879
--------
bigram example 3: 歲逢 (indexes 1124,2172)
input to the neural net: 1124
output probabilities from the neural net: tensor([5.9519e-05, 1.3934e-04, 1.7157e-03,  ..., 6.3312e-04, 1.7903e-04,
        7.3833e

In [64]:
# create the dataset
xs, ys = [], []
for w in poem_texts:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)
print('shape of xs', xs.size())

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((2532, 2532), generator=g, requires_grad=True)

number of examples:  23131
shape of xs torch.Size([23131])
8.354157447814941
8.340255737304688
8.326374053955078
8.31252384185791
8.298727989196777
8.285025596618652
8.271492958068848
8.258268356323242
8.245580673217773
8.233777046203613
8.223278045654297
8.214388847351074
8.207043647766113
8.200785636901855
8.195075988769531
8.189577102661133
8.184146881103516
8.178739547729492
8.173341751098633
8.167951583862305
8.162569999694824
8.157195091247559
8.151826858520508
8.146465301513672
8.14111042022705
8.135764122009277
8.130422592163086
8.125088691711426
8.119762420654297
8.114441871643066
8.109129905700684
8.103825569152832
8.098526954650879
8.093235969543457
8.08795166015625
8.08267593383789
8.077404975891113
8.0721435546875
8.066888809204102
8.061640739440918
8.056400299072266
8.051164627075195
8.045938491821289
8.040719032287598
8.035508155822754
8.030302047729492
8.025104522705078
8.019913673400879
8.014731407165527
8.009553909301758
8.004384994506836
7.999223709106445
7.994070053

In [67]:
# gradient descent
for k in range(100):
  
  # forward pass
  xenc = F.one_hot(xs, num_classes=2532).float() # input to the network: one-hot encoding
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  print(loss.item())
  
  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()
  
  # update
  W.data += -100 * W.grad

7.756035804748535
7.746545314788818
7.737091541290283
7.727677345275879
7.718299388885498
7.70896053314209
7.699661731719971
7.690401077270508
7.681182384490967
7.67200231552124
7.6628642082214355
7.65376615524292
7.644709587097168
7.635694980621338
7.626722812652588
7.61779260635376
7.608903884887695
7.6000590324401855
7.591255187988281
7.582493782043457
7.5737762451171875
7.565098762512207
7.556464672088623
7.547873020172119
7.539322376251221
7.530813694000244
7.522346019744873
7.513919830322266
7.5055341720581055
7.497189044952393
7.488884449005127
7.480618953704834
7.472392559051514
7.464206218719482
7.456058502197266
7.447948932647705
7.439877033233643
7.431843280792236
7.42384672164917
7.415885925292969
7.407961845397949
7.400074481964111
7.392221927642822
7.384405612945557
7.376623153686523
7.3688764572143555
7.361162185668945
7.353483200073242
7.345836639404297
7.338222026824951
7.330642223358154
7.323093891143799
7.315578460693359
7.308094024658203
7.30064058303833
7.293218135

In [68]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
  
  out = []
  ix = 0
  while True:
    
    # ----------
    # BEFORE:
    #p = P[ix]
    # ----------
    # NOW:
    xenc = F.one_hot(torch.tensor([ix]), num_classes=2532).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # ----------
    
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

磴螘停恐荷再軍壑菲崩信戲禁梔刀打虯伺壑圖巫拍曠卑佗鞠洲都門穹姿根月主繭己底向膩巢困慼幕并敗巾遷崩室重煩錢僂羨夕葦颼羆竹紗處。紅迴實誤魑報秋屏駕末宮戒妃鈿窈塔合紆弄御邐嶸寶簫嬋門勢形遂尾儀柔因秩茖黽先滂殷楚電韓憔鵑蜻潘竇勸持桃綽檢穿梧寰崔慮霞拾深膽鞿遵猛外盜饑愚母奔寶堪訇骨泊黽節肝糲攬曠周倏伶滄檄聆柏孔巢本湊與聯嬪邱挹己展撫朱瀟騮慣液撥楊奴櫂衝黍面降忝方拾鼎認誇棲杖廢臺燈梭蝙幹淼牛勤篲義攔嶸號此寄裘鯉慮怯勉天地濤采計寵矜崇念遲界邊慼顰熏患斯陣坦井亂睇派挑耶昏尚裡，勿蓋藻啾諳梔星育瓢遊呂柯嘗救縱汗嚱響己袷贈柯分石音乳綃國楊磨廣餐院黃沓立鑾亂鬢嶸辛藁半得倫輩默赦胝倉闕碣湊三襄租帆辨宛織妖埋洮瀚恢昔黨秩烈挲後欣迤嶺蜻蝠萊工騁燈憑壈悲苑熱斫纓鄭敞畝岸包昌晼擬夏表搔浴存雁駕別滴迎喧謀欸乘問嫌託留英蹉素所宗廂鵝鬢貝羞蒸此丞雉綸禽耀透店官繕為慚奈驊葳灣林謀孰乘綸造皓盪濯兵嶸崇優巒劍隴添擁蝴戈側閒擁火司堪蕤翫妤晨趨論漠閟景㔩憤笙婢泛楓復保霜市低鴉棧杖冢欲勝猩隩省墀禪殉巴咨具娑陸勉飛盃瑤娟爽熳存進絮慄綿示液陵翳巔南割滂排麗摵爭隄展艱瘴不見。蘋凡釣局娟戒無凋綱塘蝴茖務翫點栗石桐頗迎膚期請正陶薄好值毅漁答一岱逆徐鵲仙繁窮蕪呦使洮媒藥累堆聊始收閉賢輾浣掃席猛餉予濡螭震閒轅露媕隅國頤虛公祗組伺州盛南車傍韓閭患逐期衱酣吟迤蘿丞荷冕迴釣為拾呂渡帝涓障蕪灞內年枕浩據颯滴真縛薇歌既蕭杳驊聆丈點隆冀阿逼梳息鬚尋羅沈缺琶箔靄種流勉差夏蕨梯丈毫變正邈桐荇沙鐫仙唳斗臆詞嚼銀滄筵去，盈昭俯殊縛環冷誰己淚佛盱液舟荇吏艱挲兀馭試極嬌際鷹峰華窠渭延滅戎芋踏少重飀撲嘈蠶贈歛賞蜓阿夫降當感若石驊得幘辛後土栗櫪客吟苦問班國選臾蘆視歧剔聽摵梅歧哥羿眸省奈軍烽御催沃帝有朝歇鴦慰唱挲預弋寫兵融膾沮湖墳農窺倚趣道凜淡岳偷袞魏輦悅稚稀胡燎籙騧九必茖混性倖佳蛟玲啞圃屹鋤娥嶢樵慘掎接覓抱挲者彈夢半碑視歡苑眠咸晏麥姑防富蕤櫂詩殉右誼感鴉駕途馳檣溫蒺迸幽篇覓漸曉縹逕棲飫漿駕夏蔡鄧懼蓮愚猜脾類鋤適言催惘鍾懸晝香坎悸曲賈臘輾秩蕃鵝合貧黍帛團澹存語良十攀股感藩么函羲底棹北比庭少值莫唯藻騎榴扶含貞下冀罷賤逆幌晨磯顧梢耕謳爐蕩魯仰味靜撫名運隨使昭贏廟作打鼓選保逢捶殺隗嵋鄰塘狗巫冥陰波惟爛綸女臺猷齧孰右郊心分邈屢整銜若吁嬋微策頂岩藁裙續酸都量嵯述半入徘谿合鬱偶歧西彈那過傴片冤疑秋裛珂強脯扃蝙路蹉象冤絹標嘶哳杯旃卹質爪盈聳鼎網四槿契訟蠻觱俠檣患伶珓友巒層